In [ ]:
from math import sqrt, exp, erf
import numpy as np

def Sub_InitialField_SubLoop(i0, j0, k0, XI, YJ, ZK, model_EC, dx, dy, dz, subloop_x, subloop_y, L_subloop, Alpha):  
    pi = 3.141592653589793
    Permeability_vac = 1.256637061435917*10**-6
    dt = np.empty((2))
    EX_subloop = np.empty((XI, YJ+1, ZK+1, 2))
    EY_subloop = np.empty((XI+1, YJ, ZK+1, 2))
    EZ_subloop = np.empty((XI+1, YJ+1, ZK, 2))
    HX_subloop = np.empty((XI+1, YJ, ZK, 2))
    HY_subloop = np.empty((XI, YJ+1, ZK, 2))
    HZ_subloop = np.empty((XI, YJ, ZK+1, 2))

    initialfield_EC = model_EC[i0-1, j0-1, k0]
    Txmm = L_subloop*L_subloop
    t0_E = 1.13*Permeability_vac*model_EC[i0-1, j0-1, k0-1]*dz[k0-1]**2
    dt[0] = Alpha*dz[k0-1] *         sqrt(Permeability_vac*model_EC[i0-1, j0-1, k0-1]*t0_E/6)
    t0_H = t0_E+dt[0]/2

    for k in range(1, ZK):
        for j in range(1, YJ):
            for i in range(0, XI):
                coord_x = coor_x_calc(i, 1, i0, dx)+subloop_x
                coord_y = coor_y_calc(j, 1, j0, dy)+subloop_y
                coord_z = coor_z_calc(k, 1, k0, dz)
                coord_r = sqrt(coord_x**2+coord_y**2+coord_z**2)
                u = coord_r*sqrt(2*pi*initialfield_EC/(10**7*t0_E))
                if u > 25:
                    expu2 = float('inf')
                else:
                    expu2 = exp(u**2/2)
                EX_subloop[i, j, k, 0] = sqrt(2/pi)*Txmm*coord_y*u**5/(4*pi*coord_r**5*initialfield_EC*expu2)
    # Calculate subloop initial field Ey:
    for k in range(1, ZK):
        for j in range(0, YJ):
            for i in range(1, XI):
                coord_x = coor_x_calc(i, 2, i0, dx)+subloop_x
                coord_y = coor_y_calc(j, 2, j0, dy)+subloop_y
                coord_z = coor_z_calc(k, 2, k0, dz)
                coord_r = sqrt(coord_x**2+coord_y**2+coord_z**2)
                u = coord_r*sqrt(2*pi*initialfield_EC/(10**7*t0_E))
                if u > 25:
                    expu2 = float('inf')
                else:
                    expu2 = exp(u**2/2)
                EY_subloop[i, j, k, 0] = sqrt(2/pi)*Txmm*coord_x*u**5/(4*pi*coord_r**5*initialfield_EC*expu2)

    # Calculate subloop initial field Ez:
    EZ_subloop[1:XI, 1:YJ, 0:ZK, 0] = 0
    # Calculate subloop initial field Hx:
    for k in range(0, ZK):
        for j in range(0, YJ):
            for i in range(0, XI+1):
                coord_x = coor_x_calc(i, 4, i0, dx)+subloop_x
                coord_y = coor_y_calc(j, 4, j0, dy)+subloop_y
                coord_z = coor_z_calc(k, 4, k0, dz)
                coord_r = sqrt(coord_x**2+coord_y**2+coord_z**2)
                u = coord_r*sqrt(2*pi*initialfield_EC/(10**7*t0_H))
                if u > 25:
                    expu2 = float('inf')
                else:
                    expu2 = exp(u**2/2)
                HX_subloop[i, j, k, 0] = Txmm*coord_z*coord_x/(4*pi*coord_r**5)*(3*erf(u)-sqrt(2/pi)*u*(3+u**2)/expu2)

    # Calculate subloop initial field Hy:
    for k in range(0, ZK):
        for j in range(0, YJ+1):
            for i in range(0, XI):
                coord_x = coor_x_calc(i, 5, i0, dx)+subloop_x
                coord_y = coor_y_calc(j, 5, j0, dy)+subloop_y
                coord_z = coor_z_calc(k, 5, k0, dz)
                coord_r = sqrt(coord_x**2+coord_y**2+coord_z**2)
                u = coord_r*sqrt(2*pi*initialfield_EC/(10**7*t0_H))
                if u > 25:
                    expu2 = float('inf')
                else:
                    expu2 = exp(u**2/2)
                HY_subloop[i, j, k, 0] = Txmm*coord_z*coord_y/(4*pi*coord_r**5)*(3*erf(u)-sqrt(2/pi)*u*(3+u**2)/expu2)

    # Calculate subloop initial field Hz:
    for k in range(0, ZK+1):
        for j in range(0, YJ):
            for i in range(0, XI):
                coord_x = coor_x_calc(i, 6, i0, dx)+subloop_x
                coord_y = coor_y_calc(j, 6, j0, dy)+subloop_y
                coord_z = coor_z_calc(k, 6, k0, dz)
                coord_r = sqrt(coord_x**2+coord_y**2+coord_z**2)
                u = coord_r*sqrt(2*pi*initialfield_EC/(10**7*t0_H))
                if u > 25:
                    expu2 = float('inf')
                else:
                    expu2 = exp(u**2/2)
                if coord_r < 0.001*dz[k0]:
                    coord_r = 0.001*dz[k0] # Avoid singularity.
                HZ_subloop[i, j, k, 0] = Txmm/(4*pi*coord_r**5)*((2*coord_z**2-sqrt(coord_x**2+coord_y**2))*erf(u)
                       - (2*coord_z**2-sqrt(coord_x**2+coord_y**2)*(1+u**2))
                       * sqrt(2/pi)/expu2)
    return(EX_subloop, EY_subloop, EZ_subloop, HX_subloop, HY_subloop, HZ_subloop, t0_E, t0_H)

def coor_x_calc(x, m,i0,dx):
    coor_x=0
    if (m == 2 or m == 3 or m == 4):
        if (x <= i0-1):
            for i in range(x, i0):
                coor_x=coor_x+dx[i]
            coor_x=coor_x-0.5*dx[i0-1]
            coor_x=-coor_x
        elif(x > i0-1):
            for i in range(i0-1, x):
                coor_x=coor_x+dx[i]
            coor_x=coor_x-0.5*dx[i0-1]
    else:
        if (x <= i0-1):
            for i in range(x, i0):
                coor_x=coor_x+dx[i]
            coor_x=coor_x-0.5*(dx[x]+dx[i0-1])
            coor_x=-coor_x
        elif(x > i0-1):
            for i in range(i0-1, x+1):
                coor_x=coor_x+dx[i]
            coor_x=coor_x-0.5*(dx[i0-1]+dx[x])
    return coor_x

def coor_y_calc(y, m,j0,dy):
    coor_y = 0
    if (m == 1 or m == 3 or m == 5):
        if (y <= j0-1):
            for i in range(y,j0):
                coor_y = coor_y+dy[i]
            coor_y = coor_y-0.5*dy[j0-1]
            coor_y = -coor_y
        elif(y > j0-1):
            for i in range(j0-1,y):
                coor_y = coor_y+dy[i]
            coor_y = coor_y-0.5*dy[j0-1]
    else:
        if (y <= j0-1):
            for i in range(y,j0):
                coor_y = coor_y+dy[i]
            coor_y = coor_y-0.5*(dy[y]+dy[j0-1])
            coor_y = -coor_y
        elif(y > j0-1):
            for i in range(j0-1,y+1):
                coor_y = coor_y+dy[i]
            coor_y = coor_y-0.5*(dy[j0-1]+dy[y])
    return coor_y        

def coor_z_calc(z, m,k0,dz):
    coor_z = 0
    if (m == 3 or m == 4 or m == 5):
        if (z <= k0-1):
            for i in range(z,k0):
                coor_z = coor_z+dz[i]
            coor_z = coor_z-0.5*dz[z]
            coor_z = -coor_z
        elif(z > k0-1):
            for i in range(k0,z+1):
                coor_z = coor_z+dz[i]
            coor_z = coor_z-0.5*dz[z]
    else:
        if (z <= k0-1):
            for i in range(z,k0):
                coor_z = coor_z+dz[i]
            coor_z = -coor_z
        elif(z > k0-1):
            for i in range(k0,z):
                coor_z = coor_z+dz[i]      
    return coor_z      